In [18]:
import QuantLib as ql
import numpy as np
from timeit import default_timer as timer

import numpy as np
from timeit import default_timer as timer

class BasketOption:
    def __init__(self, calculation_date, maturity, strike_price, risk_free_rate, option_type, options_number, options_raw):
        self.maturity = maturity
        self.options_number = options_number
        self.options_raw = options_raw
        self.strike_price = strike_price
        self.risk_free_rate = risk_free_rate
        self.option_type = option_type
        self.calculation_date = calculation_date

    def engine_price(self):
        # global data
        ql.Settings.instance().evaluationDate = self.calculation_date
        riskFreeRate = ql.FlatForward(self.calculation_date, self.risk_free_rate, ql.Actual365Fixed())

        # option parameters
        payoff = ql.PlainVanillaPayoff(self.option_type, self.strike_price)
        exercise = ql.EuropeanExercise(self.maturity)
        day_count = ql.Actual365Fixed()
        calendar = ql.UnitedStates()
        
        # market data
        options = []
        processes = []
        for i in range(self.options_number):
            option = {}
            option['stock_price'] = ql.SimpleQuote(self.options_raw[i][0])
            option['volatility'] = ql.BlackConstantVol(self.calculation_date, calendar, self.options_raw[i][1], ql.Actual365Fixed())
            option['dividend'] = ql.FlatForward(self.calculation_date,self.options_raw[i][2], ql.Actual365Fixed())
            option['process'] = ql.BlackScholesMertonProcess(
                        ql.QuoteHandle(option['stock_price']),
                        ql.YieldTermStructureHandle(option['dividend']),
                        ql.YieldTermStructureHandle(riskFreeRate),
                        ql.BlackVolTermStructureHandle(option['volatility']),
                    )
            processes.append(option['process'])
            options.append(option)
            
        matrix = ql.Matrix(self.options_number, self.options_number)
        for j in range(self.options_number):
            for k in range(self.options_number):
                if j == k:
                    matrix[j][k] = 1.0
                else:
                    matrix[j][k] = 0.5

        process = ql.StochasticProcessArray(processes, matrix)

        basketoption = ql.BasketOption(ql.MaxBasketPayoff(payoff), exercise)
        basketoption.setPricingEngine(
            ql.MCEuropeanBasketEngine(process, "pseudorandom", timeStepsPerYear=1, requiredTolerance=0.02, seed=42)
        )
        print(basketoption.NPV())
        
        basketoption = ql.BasketOption(ql.MinBasketPayoff(payoff), exercise)
        basketoption.setPricingEngine(
            ql.MCEuropeanBasketEngine(process, "pseudorandom", timeStepsPerYear=1, requiredTolerance=0.02, seed=42)
        )
        print(basketoption.NPV())

        basketoption = ql.BasketOption(ql.AverageBasketPayoff(payoff, 2), exercise)
        basketoption.setPricingEngine(
                ql.MCEuropeanBasketEngine(
                process,
                "pseudorandom",
                timeSteps=10000,
                requiredTolerance=0.02,
                seed=42
            )
        )
        print(basketoption.NPV())


In [22]:
options_number = 3
options_raw = [[7, 0.1, 0.05],[7, 0.1, 0.05], [7, 0.1, 0.05]]
calculation_date = ql.Date(15, ql.May, 1998)
maturity = ql.Date(17, ql.May, 1999)
option_type = ql.Option.Call

sample_option = BasketOption(calculation_date, maturity, 8.0, 0.05, option_type, options_number, options_raw)
sample_option.engine_price()

0.07405418453778291
0.0022840212437264347
0.020653984243541246
